In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\new smote\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=20)

# Save the trained model
model.save('lung_detection_dblstmnotpre20.h5')

Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 656s 1s/step - accuracy: 0.4795 - loss: 1.3349
Epoch 2/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 623s 1s/step - accuracy: 0.7434 - loss: 0.7010
Epoch 3/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 681s 1s/step - accuracy: 0.8087 - loss: 0.5084
Epoch 4/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 626s 1s/step - accuracy: 0.8670 - loss: 0.3596
Epoch 5/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step - accuracy: 0.9092 - loss: 0.2515
Epoch 6/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 624s 1s/step - accuracy: 0.9381 - loss: 0.1836
Epoch 7/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 618s 1s/step - accuracy: 0.9510 - loss: 0.1384
Epoch 8/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 592s 1s/step - accuracy: 0.9579 - loss: 0.1268
Epoch 9/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 595s 1s/step - accuracy: 0.9690 - loss: 0.0869
Epoch 10/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 600s 1s/step - accuracy: 0.9726 - loss: 0.0873
Epoch 11/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 586s 1s/step - accuracy: 0.9814 - loss: 0.0507
Epoch 12/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set the path to your test dataset
test_data_dir = r"D:\new smote\test-20230326T155708Z-001\test"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Load the trained model
model = load_model('lung_detection_dblstmnotpre20.h5')

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Predict probabilities for each class
y_pred_probabilities = model.predict(X_test_reshaped)

# Get the predicted class indices
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy}")

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)


Found 4110 images belonging to 6 classes.


129/129 ━━━━━━━━━━━━━━━━━━━━ 17s 124ms/step
Test Accuracy: 0.7858880778588808
Confusion Matrix:
[[567  47   2   3  52  14]
 [ 47 511  19   3  66  39]
 [ 23  18 547  16  74   7]
 [ 15  22  65 565  14   4]
 [ 67  77  82   5 432  22]
 [  9  23  10   4  31 608]]
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.78      0.83      0.80       685
       COVID       0.73      0.75      0.74       685
      NORMAL       0.75      0.80      0.78       685
   PNEUMONIA       0.95      0.82      0.88       685
PNEUMOTHORAX       0.65      0.63      0.64       685
TUBERCULOSIS       0.88      0.89      0.88       685

    accuracy                           0.79      4110
   macro avg       0.79      0.79      0.79      4110
weighted avg       0.79      0.79      0.79      4110



In [35]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_dblstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\asuto\Desktop\intern ship 2\lung detection\archive\archive\val-20230326T152503Z-001\val\NORMAL\00000006_000.png"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = np.expand_dims(img_array, axis=1)  # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 400ms/step
Predicted class: NORMAL


In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\new smote\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    steps_per_epoch=312,
    epochs=30)

# Save the trained model
model.save('lung_detection_dblstmnotpre30.h5')

Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.4688 - loss: 1.3219
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 196s 628ms/step - accuracy: 0.7006 - loss: 0.8153
Epoch 3/30


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.7552 - loss: 0.6687
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 231s 740ms/step - accuracy: 0.7914 - loss: 0.5806
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 393s 1s/step - accuracy: 0.8387 - loss: 0.4644
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 253s 810ms/step - accuracy: 0.8404 - loss: 0.4328
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 419s 1s/step - accuracy: 0.8971 - loss: 0.2821
Epoch 8/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 255s 815ms/step - accuracy: 0.8891 - loss: 0.3104
Epoch 9/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 553s 2s/step - accuracy: 0.9277 - loss: 0.2056
Epoch 10/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 366s 1s/step - accuracy: 0.9220 - loss: 0.2062
Epoch 11/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 594s 2s/step - accuracy: 0.9524 - loss: 0.1403
Epoch 12/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 365s 1s/step - accuracy: 0.9297 - loss: 0.1832
Epoch 13/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 549s 2s/step - accuracy: 0.9603 - loss: 0.1131
Epoch 14/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set the path to your test dataset
test_data_dir = r"D:\new smote\test-20230326T155708Z-001\test"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Load the trained model
model = load_model('lung_detection_dblstmnotpre30.h5')

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Predict probabilities for each class
y_pred_probabilities = model.predict(X_test_reshaped)

# Get the predicted class indices
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy}")

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)


Found 4110 images belonging to 6 classes.


129/129 ━━━━━━━━━━━━━━━━━━━━ 32s 232ms/step
Test Accuracy: 0.7875912408759124
Confusion Matrix:
[[558  28   3   5  87   4]
 [ 43 463  26  12 100  41]
 [ 15  21 533  26  85   5]
 [  5   4  49 599  23   5]
 [ 32  64  71  11 490  17]
 [  9  19   6   3  54 594]]
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.84      0.81      0.83       685
       COVID       0.77      0.68      0.72       685
      NORMAL       0.77      0.78      0.78       685
   PNEUMONIA       0.91      0.87      0.89       685
PNEUMOTHORAX       0.58      0.72      0.64       685
TUBERCULOSIS       0.89      0.87      0.88       685

    accuracy                           0.79      4110
   macro avg       0.80      0.79      0.79      4110
weighted avg       0.80      0.79      0.79      4110



In [3]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\new smote\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the DBLSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    steps_per_epoch=312,
    epochs=10)

# Save the trained model
model.save('lung_detection_dblstmnotpre10.h5')

Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 393s 1s/step - accuracy: 0.4482 - loss: 1.3655
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 215s 686ms/step - accuracy: 0.6654 - loss: 0.9000
Epoch 3/10


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 386s 1s/step - accuracy: 0.7371 - loss: 0.7258
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 211s 674ms/step - accuracy: 0.7507 - loss: 0.6678
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 353s 1s/step - accuracy: 0.8110 - loss: 0.5258
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 213s 683ms/step - accuracy: 0.8081 - loss: 0.5267
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 399s 1s/step - accuracy: 0.8587 - loss: 0.3919
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 208s 666ms/step - accuracy: 0.8590 - loss: 0.3817
Epoch 9/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 341s 1s/step - accuracy: 0.9098 - loss: 0.2448
Epoch 10/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 209s 669ms/step - accuracy: 0.9006 - loss: 0.2725


In [4]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set the path to your test dataset
test_data_dir = r"D:\new smote\test-20230326T155708Z-001\test"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Load the trained model
model = load_model('lung_detection_dblstmnotpre30.h5')

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Predict probabilities for each class
y_pred_probabilities = model.predict(X_test_reshaped)

# Get the predicted class indices
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy}")

# Generate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(report)


Found 4110 images belonging to 6 classes.


129/129 ━━━━━━━━━━━━━━━━━━━━ 32s 238ms/step
Test Accuracy: 0.7875912408759124
Confusion Matrix:
[[558  28   3   5  87   4]
 [ 43 463  26  12 100  41]
 [ 15  21 533  26  85   5]
 [  5   4  49 599  23   5]
 [ 32  64  71  11 490  17]
 [  9  19   6   3  54 594]]
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.84      0.81      0.83       685
       COVID       0.77      0.68      0.72       685
      NORMAL       0.77      0.78      0.78       685
   PNEUMONIA       0.91      0.87      0.89       685
PNEUMOTHORAX       0.58      0.72      0.64       685
TUBERCULOSIS       0.89      0.87      0.88       685

    accuracy                           0.79      4110
   macro avg       0.80      0.79      0.79      4110
weighted avg       0.80      0.79      0.79      4110

